# Tugas-2

## 1.Web Server

In [ ]:
# import library socket
import socket as sc

# Definisikan IP address
HOST = "192.168.1.4"
# Port HTTP adalah 80
PORT = 80

# Definisikan socket dengan tipe TCP
s = sc.socket(sc.AF_INET, sc.SOCK_STREAM)

# Lakukan binding pada socket 
s.bind((HOST, PORT))

# Socket listen pada IP dan Port yang sudah didefinisikan
s.listen()

# Tampilkan bahwa server sudah siap melayani request
print("Server sudah berjalan dan siap melayani pada: {}:{}".format(HOST, PORT))

# Loop forever
while True:
    # Socket menerima request
    conn, addr = s.accept()
    
    # Socket menerima data
    RECEIVED = conn.recv(1024)
    
    # Print data hasil koneksi
    print(RECEIVED)
    
    # Buat respone HTTP untuk client
    http_response = """\HTTP/1.1 200 OK

<html>
<body style="background-color:powderblue;">

<h1 style="text-align:center;">Centered Heading</h1>
<p style="text-align:center;">Centered paragraph.</p>

<p>I am normal</p>
<p style="color:red;">I am red</p>
<p style="color:blue;">I am blue</p>
<p style="font-size:50px;">I am big</p>

</body>
</html>
"""
    # Kirim response kepada client dengan sendall() jangan lupa diencode response dengan utf-8 
    conn.sendall(http_response.encode('utf-8'))
    
    # Tutup koneksi
    conn.close()


## 2.Sistem Voting

### 2.1.Server

In [ ]:
# Import library SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer

# import library SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCRequestHandler

# Definisikan IP address dan port
IP = "192.168.1.4"
PORT = 8080

# Batasi path (folder) yang bisa diakses
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)
    
# Buat server
with SimpleXMLRPCServer((IP, PORT),
                        requestHandler=RequestHandler) as server:
    server.register_introspection_functions()

    # Data struktur dictionary kandidat voting
    candidate = {    
        "Burhan": 0,
        "Jamal": 0,
        "Reinhard": 0
    }
    
    # Fungsi untuk vote kandidat dengan input nama kandidatnya
    def vote_add(name: str) -> str:
        # jika kandidat ada dalam dictionary maka tambahkan nilai votenya
        if name in candidate:
            candidate[name] += 1
            pesan = "Hatur nuhun, voting untuk {} telah diterima dengan selamat sentosa".format(name)
        else:
            pesan =  "Kandidat dengan nama {} tidak terdaftar".format(name)
        return pesan
    
    # Registrasi fungsi vote_add() ke server
    server.register_function(vote_add, 'vote')

    # Fungsi untuk menampilkan hasil vote dari semua kandidat
    def vote_result() -> str:        
        # Hitung total vote
        total_vote = sum([x for x in candidate.values()])
        
        persentase = 'Hasil voting = '
        
        # Jika belum ada voting
        if total_vote == 0:
            return 'Belum ada voting yang dilakukan'
        
        # hitung hasil persentase masing-masing kandidat
        for i in candidate:
            persentase += i+ ' ' + str(candidate[i]/total_vote*100)+'%, '
            
        return persentase
        
    # Registrasi fungsi vote_result() ke server
    server.register_function(vote_result)

    print ("Server no voting ga starto desuyooo....")
    # Server START
    server.serve_forever()

### 2.2.Client

In [13]:
# Import xmlrpc untuk client
import xmlrpc.client

# Definisikan IP address dan port
IP = "192.168.1.4"
PORT = 8080

# Buat sebuah proxy untuk mengakses server
sp = xmlrpc.client.ServerProxy(f'http://{IP}:{PORT}')

# Print daftar method yang tersedia di server
print(sp.system.listMethods())

['system.listMethods', 'system.methodHelp', 'system.methodSignature', 'vote', 'vote_result']


In [14]:
# Print hasil voting
print(sp.vote_result())

Burhan 50.0%, Jamal 25.0%, Reinhard 25.0%, 


In [15]:
# Voting
print(sp.vote('Burhan'))
print(sp.vote('Jamal'))
print(sp.vote('Reinhard'))
print(sp.vote('Burhan'))

# Voting yang salah
print(sp.vote('Taufik'))

# Print hasil voting
print(sp.vote_result())

Hatur nuhun, voting untuk Burhan telah diterima dengan selamat sentosa
Hatur nuhun, voting untuk Jamal telah diterima dengan selamat sentosa
Hatur nuhun, voting untuk Reinhard telah diterima dengan selamat sentosa
Hatur nuhun, voting untuk Burhan telah diterima dengan selamat sentosa
Kandidat dengan nama Taufik tidak terdaftar
Burhan 50.0%, Jamal 25.0%, Reinhard 25.0%, 


# 3.